## 라이브러리 불러오기

## 데이터 불러오기

## ID, ZIP.Code 컬럼 삭제

## 목표변수 범주 확인

In [ ]:
# 시각화


## train / test split

## 의사결정나무 모델 생성
### sklearn update
### https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# class_weight : class별 가중치
# criterion : 가지치기 기준
# max_depth : 최대 깊이
# max_features : 사용하는 최대 column 개수(auto ,sqrt, log2)
# max_leaf_node : 최대 leaf 노드의 개수
# min_samples_split : node를 split할 때 중간 노드에 최소 있어야하는 데이터의 개수
# min_samples_leaf : node를 split할 때 리프 노드에 최소 있어야하는 데이터의 개수
# ccp_alpha : 과적합 방지를 위한 손실함수 parameter



In [ ]:
# 모델 학습


In [ ]:
# 예측
